In [1]:
import tensorflow as tf

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier

import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')

def accuracy_function(tp, tn, fp, fn):
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    return accuracy


def precision_function(tp, fp):
    precision = tp / (tp+fp)
    return precision


def recall_function(tp, fn):
    recall = tp / (tp+fn)
    return recall

def confusion_matrix(truth, predicted):

    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0

    for true, pred in zip(truth, predicted):

        if true == '1' or 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1

        elif true == '0' or 1:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1

    accuracy = accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision = precision_function(true_positive, false_positive)
    recall = recall_function(true_positive, false_negative)

    return accuracy, precision, recall

def build_cnn():

    model = tf.keras.Sequential(name="CNN")

    model.add(tf.keras.layers.Input(shape=(4096, 1)))

    model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=4, activation='relu'))
    model.add(tf.keras.layers.MaxPooling1D(pool_size=2))

    model.add(tf.keras.layers.Conv1D(filters=128, kernel_size=4, activation='relu'))
    model.add(tf.keras.layers.MaxPooling1D(pool_size=2))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    print(model.summary())

    return model

if __name__ == '__main__':
    df = pd.read_csv("./dataset/SQLiV3.csv")

    # Drop duplicate row
    df = df.drop_duplicates('Sentence')

    df = df.drop('Unnamed: 2', axis=1, errors='ignore')
    df = df.drop('Unnamed: 3', axis=1, errors='ignore')

    # Drop row with Nan
    df = df.dropna(how='any')

    # Drop row with incorrect label
    df = df[(df['Label'] == '0') | (df['Label'] == '1')]

    print(df.head())

    # Reset index
    df = df.sample(frac=1).reset_index(drop=True)
    y = np.array([int(i) for i in df['Label'].values])

    vectorizer = CountVectorizer(max_features=4096, ngram_range=(2, 2))
    #vectorizer = TfidfVectorizer(min_df=2, max_df=0.7, stop_words='english')

    count_matrix = vectorizer.fit_transform(df['Sentence']).toarray()

    X_train, X_test, y_train, y_test = train_test_split(count_matrix, y, test_size=0.2, random_state=42)

    # (sentence_index, feature_index) count
    print("Train shape: {}".format(X_train.shape))
    print("Test shape: {}".format(X_test.shape))

    # Logistic Regression
    model = LogisticRegression().fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy, precision, recall = confusion_matrix(y_test, y_pred)
    print("=================Logistic Regression Result=================")
    print("Accuracy : {:.4f}".format(accuracy))
    print("Precision : {:.4f}".format(precision))
    print("Recall : {:.4f}".format(recall))


    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy, precision, recall = confusion_matrix(y_test, y_pred)
    print("=================KNN Result=================")
    print("Accuracy : {:.4f}".format(accuracy))
    print("Precision : {:.4f}".format(precision))
    print("Recall : {:.4f}".format(recall))

    X_train = X_train.reshape(-1, 4096, 1)
    X_test = X_test.reshape(-1, 4096, 1)

    # CNN shape
    print("Train shape: {}".format(X_train.shape))
    print("Test shape: {}".format(X_test.shape))

    cnn = build_cnn()
    cnn.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4), metrics=['accuracy'])
    cnn.fit(X_train, y_train, batch_size=10, epochs=5, validation_data=(X_test, y_test))

    y_pred = cnn.predict(X_test)

    # Turn sigmoid result to 0 or 1
    y_pred = np.where(y_pred > 0.5, 1, 0)
    
    accuracy, precision, recall = confusion_matrix(y_test, y_pred)
    print("=================CNN Result=================")
    print("Accuracy : {:.4f}".format(accuracy))
    print("Precision : {:.4f}".format(precision))
    print("Recall : {:.4f}".format(recall))

2023-11-18 19:44:14.441770: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-18 19:44:14.441822: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-18 19:44:14.443400: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-18 19:44:14.548509: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package stopwords to /home

                                            Sentence Label
0                  " or pg_sleep  (  __TIME__  )  --     1
2   AND 1  =  utl_inaddr.get_host_address   (    ...     1
3   select * from users where id  =  '1' or @ @1 ...     1
4   select * from users where id  =  1 or 1#"  ( ...     1
5   select name from syscolumns where id   =     ...     1
Train shape: (24472, 4096)
Test shape: (6118, 4096)
=================Logistic Regression Result=================
Accuracy : 0.9363
Precision : 1.0000
Recall : 0.9363
=================KNN Result=================
Accuracy : 0.9364
Precision : 1.0000
Recall : 0.9364
Train shape: (24472, 4096, 1)
Test shape: (6118, 4096, 1)
Model: "CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 4093, 64)          320       
                                                                 
 max_pooling1d (MaxPooling1  (None, 2046, 64)  

2023-11-18 19:44:41.811000: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-18 19:44:41.834979: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-18 19:44:41.835013: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-18 19:44:41.838601: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-18 19:44:41.838638: I tensorflow/compile

None


2023-11-18 19:44:43.123082: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 801898496 exceeds 10% of free system memory.
2023-11-18 19:44:43.363307: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 801898496 exceeds 10% of free system memory.


Epoch 1/5


2023-11-18 19:44:44.465887: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-11-18 19:44:45.478898: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f04a0337d40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-18 19:44:45.478940: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2070, Compute Capability 7.5
2023-11-18 19:44:45.487618: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-18 19:44:45.581475: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2448/2448 [==============================] - 31s 12ms/step - loss: 0.1675 - accuracy: 0.9411 - val_loss: 0.1561 - val_accuracy: 0.9456
Epoch 2/5
2448/2448 [==============================] - 28s 11ms/step - loss: 0.1449 - accuracy: 0.9483 - val_loss: 0.1542 - val_accuracy: 0.9469
Epoch 3/5
2448/2448 [==============================] - 28s 12ms/step - loss: 0.1419 - accuracy: 0.9488 - val_loss: 0.1529 - val_accuracy: 0.9470
Epoch 4/5
2448/2448 [==============================] - 30s 12ms/step - loss: 0.1409 - accuracy: 0.9492 - val_loss: 0.1496 - val_accuracy: 0.9469
Epoch 5/5
192/192 [==============================] - 1s 5ms/step
=================CNN Result=================
Accuracy : 0.9470
Precision : 1.0000
Recall : 0.9470
